In [1]:
import os
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
def read_parquet_by_type(train_path: str):
    # Lấy tất cả các file parquet trong thư mục
    files = [os.path.join(train_path, f) for f in os.listdir(train_path) if f.endswith('.parquet')]
    
    # Phân loại các file theo loại tên
    user_chunk_files = [file for file in files if 'user_chunk' in file]
    purchase_history_chunk_files = [file for file in files if 'purchase_history_daily_chunk' in file]
    item_chunk_files = [file for file in files if 'item_chunk' in file]
    
    # Đọc các file riêng biệt thành DataFrame
    user_chunk_df = pl.concat([pl.read_parquet(file) for file in user_chunk_files]) if user_chunk_files else None
    purchase_history_chunk_df = pl.concat([pl.read_parquet(file) for file in purchase_history_chunk_files]) if purchase_history_chunk_files else None
    item_chunk_df = pl.concat([pl.read_parquet(file) for file in item_chunk_files]) if item_chunk_files else None
    
    # Trả về một dictionary chứa các DataFrame
    return {
        "user_chunk": user_chunk_df,
        "purchase_history_chunk": purchase_history_chunk_df,
        "item_chunk": item_chunk_df
    }

In [4]:
train_path = 'E:/KHMT2023_CS_UIT/05_C_Python_For_ML/recommendation_dataset'
dataframes = read_parquet_by_type(train_path)

In [33]:
df_user = dataframes["user_chunk"]
df_purchase = dataframes["purchase_history_chunk"]
df_item = dataframes["item_chunk"]

# Task 4

# User chunk

| Cột              | Giữ / Loại bỏ       | Lý do                                                                                                               |
| ---------------- | ------------------- | ------------------------------------------------------------------------------------------------------------------- |
| `customer_id`, `user_id`    | ✅ Đặc trưng định danh / mã hóa| |
| `gender`         | ✅ Đặc trưng thống kê               | |
| `location`       | ✅       | |
| `province`       | ✅ Đặc trưng thống kê                |  |
| `membership`     | ✅ Đặc trưng thống kê               |   |
| `install_app`    | ✅ Đặc trưng thống kê               |  |

In [9]:
df_user_keep = df_user.select([
    pl.col("user_id"),
    pl.col("customer_id"),
    pl.col("gender"),
    pl.col("location"),
    pl.col("province"),
    pl.col("membership"),
    pl.col("install_app")
])

print("User DataFrame:")
print(df_user_keep.head(3))

User DataFrame:
shape: (3, 7)
┌───────────────────────┬─────────────┬────────┬──────────┬─────────────┬────────────┬─────────────┐
│ user_id               ┆ customer_id ┆ gender ┆ location ┆ province    ┆ membership ┆ install_app │
│ ---                   ┆ ---         ┆ ---    ┆ ---      ┆ ---         ┆ ---        ┆ ---         │
│ str                   ┆ i32         ┆ str    ┆ i32      ┆ str         ┆ str        ┆ str         │
╞═══════════════════════╪═════════════╪════════╪══════════╪═════════════╪════════════╪═════════════╡
│ e1e48206652bf8c279ff0 ┆ 14732       ┆ Nam    ┆ 155      ┆ Hồ Chí Minh ┆ Standard   ┆ In-Store    │
│ 206c69a80…            ┆             ┆        ┆          ┆             ┆            ┆             │
│ 77891759204bd27e69fb1 ┆ 15126       ┆ Nữ     ┆ 300      ┆ Hồ Chí Minh ┆ Standard   ┆ In-Store    │
│ 1a7b92889…            ┆             ┆        ┆          ┆             ┆            ┆             │
│ b8041b584a0bb66553617 ┆ 29718       ┆ Nữ     ┆ 157      ┆ B

## Xử lý Province

In [10]:
# Thay thế các giá trị "Thành Phố" và "Tỉnh" trong cột 'province'
df_user_keep = df_user_keep.with_columns(
    pl.col('province')
    .str.replace('Thành Phố ', '')  # Loại bỏ "Thành Phố"
    .str.replace('Tỉnh ', '')  # Loại bỏ "Tỉnh"
    .alias('province')  # Đặt lại tên cột sau khi thay thế
)

# In các giá trị duy nhất sau khi thay thế
unique_provinces = df_user_keep['province'].unique().to_list()
print("Unique values in column 'province' after replacement:")
print(unique_provinces)

Unique values in column 'province' after replacement:
['Bình Định', 'Hưng Yên', 'Bến Tre', 'Kiên Giang', 'Bắc Kạn', 'Hà Nội', 'Quảng Ninh', 'Đắk Lắk', 'Khánh Hòa', 'Hải Dương', 'Hà Giang', 'Phú Thọ', 'Trà Vinh', 'Sơn La', 'Lai Châu', 'Vĩnh Phúc', 'Cần Thơ', 'Bắc Giang', 'Cà Mau', 'Cao Bằng', 'Sóc Trăng', 'Thái Nguyên', 'An Giang', 'Quảng Trị', 'Bình Thuận', 'Hà Tĩnh', 'Phú Yên', 'Long An', 'Nam Định', 'Lâm Đồng', 'Thái Bình', 'Lạng Sơn', 'Đồng Tháp', 'Đồng Nai', 'Hòa Bình', 'Hà Nam', 'Bạc Liêu', 'Tây Ninh', 'Bà Rịa - Vũng Tàu', 'Kon Tum', 'Bình Dương', 'Yên Bái', 'Quảng Bình', 'Tiền Giang', 'Ninh Thuận', 'Đắk Nông', 'Vĩnh Long', 'Thừa Thiên Huế', 'Hải Phòng', 'Nghệ An', 'Bắc Ninh', 'Quảng Nam', 'Tuyên Quang', 'Bình Phước', 'Hậu Giang', 'Ninh Bình', 'Thanh Hóa', 'Đà Nẵng', 'Quảng Ngãi', 'Gia Lai', 'Lào Cai', 'Hồ Chí Minh', 'Điện Biên']


# Purchase chunk

| Cột            | Giữ / Loại bỏ | Lý do                                                                                                                                             |
| -------------- | ------------- | ------------------------------------------------------------------------------------------------------------------------------------------------- |
| `timestamp`    | ✅ Đặc trưng thời gian         |  |
| `user_id`      | ✅ Đặc trưng định danh / mã hóa        |             |
| `item_id`      | ✅ Đặc trưng định danh / mã hóa         |       |
| `price`        | ✅          |                            |
| `quantity`     | ✅ Đặc trưng thống kê         |                                                  |
| `customer_id`  | ✅ Đặc trưng định danh / mã hóa         |   |
| `created_date` | ✅ Đặc trưng thời gian      ||
| `payment`      | ✅ Đặc trưng thống kê       |                 |
| `location`     | ✅         |          |
| `discount`     | ✅   Chuẩn hóa về 0 - 1        |                                                         |
| `channel`      | ✅ Đặc trưng thống kê         |           |

In [17]:
df_purchase_keep = df_purchase.select([
    pl.col("user_id"),
    pl.col("item_id"),
    pl.col("timestamp"),
    pl.col("quantity"),
    pl.col("customer_id"),
    pl.col("created_date"),
    pl.col("payment"),
    pl.col("location"),
    pl.col("discount"),
    pl.col("channel"),
    pl.col("price")
])

print("Purchase History DataFrame:")
print(df_purchase_keep.head(3))

Purchase History DataFrame:
shape: (3, 11)
┌────────────┬────────────┬───────────┬──────────┬───┬──────────┬───────────┬──────────┬───────────┐
│ user_id    ┆ item_id    ┆ timestamp ┆ quantity ┆ … ┆ location ┆ discount  ┆ channel  ┆ price     │
│ ---        ┆ ---        ┆ ---       ┆ ---      ┆   ┆ ---      ┆ ---       ┆ ---      ┆ ---       │
│ str        ┆ str        ┆ i64       ┆ i32      ┆   ┆ i32      ┆ decimal[3 ┆ str      ┆ decimal[3 │
│            ┆            ┆           ┆          ┆   ┆          ┆ 8,4]      ┆          ┆ 8,4]      │
╞════════════╪════════════╪═══════════╪══════════╪═══╪══════════╪═══════════╪══════════╪═══════════╡
│ ca12702ddf ┆ 7115000000 ┆ 173506422 ┆ 1        ┆ … ┆ 656      ┆ 0.0000    ┆ In-Store ┆ 49000.000 │
│ 55acaa9fb7 ┆ 004        ┆ 1         ┆          ┆   ┆          ┆           ┆          ┆ 0         │
│ 67e10faaa6 ┆            ┆           ┆          ┆   ┆          ┆           ┆          ┆           │
│ …          ┆            ┆           ┆         

### Xử lý discount bằng cách chuyển sang phần trăm dựa trên price

In [18]:
import polars as pl

# Giả sử df_item là DataFrame chứa các cột 'discount' và 'price'

df_purchase_keep = df_purchase_keep.with_columns(
    # Thay thế giá trị của cột 'discount' bằng discount_percentage (giá trị từ 0 đến 1)
    (pl.col("discount") / pl.col("price")).alias("discount")
)

# Kiểm tra kết quả
print(df_purchase_keep[['price', 'discount']].head())


shape: (5, 2)
┌───────────────┬───────────────┐
│ price         ┆ discount      │
│ ---           ┆ ---           │
│ decimal[38,4] ┆ decimal[38,4] │
╞═══════════════╪═══════════════╡
│ 49000.0000    ┆ 0.0000        │
│ 69000.0000    ┆ 0.0000        │
│ 75000.0000    ┆ 0.0000        │
│ 58500.0000    ┆ 0.2222        │
│ 89000.0000    ┆ 0.1124        │
└───────────────┴───────────────┘


### Xử lý Price bằng Winsorizing

In [19]:
import polars as pl

# Giả sử df_purchase là DataFrame Polars chứa cột 'price'

# Tính IQR cho 'price'
Q1_price = df_purchase_keep['price'].quantile(0.25)
Q3_price = df_purchase_keep['price'].quantile(0.75)
IQR_price = Q3_price - Q1_price

# Giới hạn cho Winsorizing 'price'
lower_limit_price = Q1_price - 1.5 * IQR_price
upper_limit_price = Q3_price + 1.5 * IQR_price

# Winsorizing 'price': Thay thế các giá trị ngoài giới hạn bằng giới hạn
df_purchase_keep = df_purchase_keep.with_columns(
    pl.when(pl.col('price') < lower_limit_price)
    .then(lower_limit_price)
    .otherwise(
        pl.when(pl.col('price') > upper_limit_price)
        .then(upper_limit_price)
        .otherwise(pl.col('price'))  # Giữ nguyên nếu trong phạm vi
    ).alias('price')
)

# Kiểm tra giá trị sau khi winsorizing
print("Summary statistics after Winsorizing (Price):")
print(df_purchase_keep.select('price').describe())


Summary statistics after Winsorizing (Price):
shape: (9, 2)
┌────────────┬───────────────┐
│ statistic  ┆ price         │
│ ---        ┆ ---           │
│ str        ┆ f64           │
╞════════════╪═══════════════╡
│ count      ┆ 3.5729825e7   │
│ null_count ┆ 0.0           │
│ mean       ┆ 157778.138713 │
│ std        ┆ 153952.175963 │
│ min        ┆ 0.0444        │
│ 25%        ┆ 48316.3914    │
│ 50%        ┆ 87000.0       │
│ 75%        ┆ 244000.0      │
│ max        ┆ 537525.4129   │
└────────────┴───────────────┘


# Item

| Cột                  | Giữ / Loại bỏ       | Lý do                                                                                                                                                    |
| -------------------- | ------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `created_date`       | ✅ Đặc trưng thời gian             |     |
| `creation_timestamp` | ✅ Đặc trưng thời gian               |  |
| `price`              | ✅             |  |
| `gp` (gross profit)  | ✅                | |
| `category`        | ✅ Đặc trưng quan trọng cho task 5               |                |
| `category_l1`        | ✅ Đặc trưng quan trọng cho task 5               |                |
| `gender_target`      | ✅ Đặc trưng cần xử lý |   |
| `sale_status`        | ✅                |      |
| `p_id`               | ✅ Đặc trưng định danh / mã hóa            |  |


In [20]:
df_item_keep = df_item.select([
    pl.col("p_id"), 
    pl.col("category"),
    pl.col("category_l1"),
    pl.col("created_date"),
    pl.col("creation_timestamp"),
    pl.col("price"),
    pl.col("gp"),
    pl.col("gender_target"),
    pl.col("sale_status")
])

### Xử lý Price bằng Winsorizing

In [21]:
import polars as pl

# Giả sử df_item là DataFrame Polars chứa cột 'price'

# Tính IQR cho 'price'
Q1_price = df_item_keep['price'].quantile(0.25)
Q3_price = df_item_keep['price'].quantile(0.75)
IQR_price = Q3_price - Q1_price

# Giới hạn cho Winsorizing 'price'
lower_limit_price = Q1_price - 1.5 * IQR_price
upper_limit_price = Q3_price + 1.5 * IQR_price

# Winsorizing 'price': Thay thế các giá trị ngoài giới hạn bằng giới hạn
df_item_keep = df_item_keep.with_columns(
    pl.when(pl.col('price') < lower_limit_price)
    .then(lower_limit_price)
    .otherwise(
        pl.when(pl.col('price') > upper_limit_price)
        .then(upper_limit_price)
        .otherwise(pl.col('price'))  # Giữ nguyên nếu trong phạm vi
    ).alias('price')
)

# Kiểm tra giá trị sau khi winsorizing
print("Summary statistics after Winsorizing (Price):")
print(df_item_keep.select('price').describe())


Summary statistics after Winsorizing (Price):
shape: (9, 2)
┌────────────┬───────────────┐
│ statistic  ┆ price         │
│ ---        ┆ ---           │
│ str        ┆ f64           │
╞════════════╪═══════════════╡
│ count      ┆ 27332.0       │
│ null_count ┆ 0.0           │
│ mean       ┆ 142751.84992  │
│ std        ┆ 109120.160588 │
│ min        ┆ 0.0           │
│ 25%        ┆ 49000.0       │
│ 50%        ┆ 119000.0      │
│ 75%        ┆ 199000.0      │
│ max        ┆ 424000.0      │
└────────────┴───────────────┘


### Xử lý gp bằng Winsorizing

In [22]:
import polars as pl

# Giả sử df_item là DataFrame Polars chứa cột 'price'

# Tính IQR cho 'price'
Q1_price = df_item_keep['gp'].quantile(0.25)
Q3_price = df_item_keep['gp'].quantile(0.75)
IQR_price = Q3_price - Q1_price

# Giới hạn cho Winsorizing 'price'
lower_limit_price = Q1_price - 1.5 * IQR_price
upper_limit_price = Q3_price + 1.5 * IQR_price

# Winsorizing 'price': Thay thế các giá trị ngoài giới hạn bằng giới hạn
df_item_keep = df_item_keep.with_columns(
    pl.when(pl.col('gp') < lower_limit_price)
    .then(lower_limit_price)
    .otherwise(
        pl.when(pl.col('gp') > upper_limit_price)
        .then(upper_limit_price)
        .otherwise(pl.col('gp'))  # Giữ nguyên nếu trong phạm vi
    ).alias('gp')
)

# Kiểm tra giá trị sau khi winsorizing
print("Summary statistics after Winsorizing (gp):")
print(df_item_keep.select('gp').describe())


Summary statistics after Winsorizing (gp):
shape: (9, 2)
┌────────────┬──────────────┐
│ statistic  ┆ gp           │
│ ---        ┆ ---          │
│ str        ┆ f64          │
╞════════════╪══════════════╡
│ count      ┆ 27332.0      │
│ null_count ┆ 0.0          │
│ mean       ┆ 38494.988837 │
│ std        ┆ 42347.432039 │
│ min        ┆ 0.0          │
│ 25%        ┆ 0.0          │
│ 50%        ┆ 25047.0      │
│ 75%        ┆ 65532.0      │
│ max        ┆ 163830.0     │
└────────────┴──────────────┘


### Xử lý một số giá trị "Không xác định của gender_target

In [34]:
import polars as pl

# Giả sử df_item chứa các cột 'description' và 'description_new'

# Định nghĩa các từ khóa tìm kiếm cho gender_target
gender_keywords = {
    "Bé Gái": ['bé gái', 'gái', 'nữ', 'women', 'girl'],
    "Bé Trai": ['bé trai', 'trai', 'nam', 'men', 'boy'],
    "Unisex": ['unisex'],
    "Sơ sinh": ['sơ sinh', 'newborn']
}

# Hàm để xác định giới tính (gender) trong text (description hoặc description_new)
def get_gender_from_keywords(text):
    # Kiểm tra các từ khóa trong text và trả về giới tính
    found_genders = set()
    for gender, keywords in gender_keywords.items():
        if any(keyword in text for keyword in keywords):
            found_genders.add(gender)
    if "Bé Gái" in found_genders and "Bé Trai" in found_genders:
        return "Unisex"  # Nếu tìm thấy cả "Bé Gái" và "Bé Trai", gán "Unisex"
    return found_genders.pop() if found_genders else "Không xác định"  # Nếu chỉ tìm thấy một, trả về giới tính, nếu không gán "Không xác định"


# Tạo cột mới gender_target từ description và description_new mà không dùng apply
def fill_gender_target(df):
    # Kiểm tra nếu gender_target là "Không xác định"
    condition = pl.col('gender_target') == 'Không xác định'

    # Kiểm tra nếu description chứa từ khóa giới tính và thay thế
    for gender, keywords in gender_keywords.items():
        condition_description = pl.col('description').str.contains('|'.join(keywords))
        condition_description_new = pl.col('description_new').str.contains('|'.join(keywords))
        
        # Cập nhật gender_target nếu có từ khóa trong description hoặc description_new
        df = df.with_columns(
            pl.when(condition)
            .then(
                pl.when(condition_description)
                .then(pl.lit(gender))  # Nếu tìm thấy từ khóa trong description
                .otherwise(
                    pl.when(condition_description_new)
                    .then(pl.lit(gender))  # Nếu tìm thấy từ khóa trong description_new
                    .otherwise(pl.lit("Không xác định"))
                )
            )
            .otherwise(pl.col('gender_target'))  # Giữ nguyên nếu không phải "Không xác định"
            .alias('gender_target')
        )
    
    return df

# Áp dụng hàm vào DataFrame
df_item = fill_gender_target(df_item)

# Kiểm tra kết quả
print(df_item[['gender_target', 'description', 'description_new']].head())


shape: (5, 3)
┌────────────────┬─────────────────────────────────┬─────────────────────────────────┐
│ gender_target  ┆ description                     ┆ description_new                 │
│ ---            ┆ ---                             ┆ ---                             │
│ str            ┆ str                             ┆ str                             │
╞════════════════╪═════════════════════════════════╪═════════════════════════════════╡
│ Không xác định ┆ Không xác định                  ┆ Chi tiết sản phẩm             … │
│ Bé Gái         ┆ Không xác định                  ┆ Không xác định                  │
│ Không xác định ┆ - Chất liệu: Sản phẩm được làm… ┆ Chi tiết sản phẩm             … │
│ Không xác định ┆ ﻿﻿Tã dán Merries size S 82 miế…   ┆ Không xác định                  │
│ Không xác định ┆ ﻿﻿﻿Bỉm tã quần Merries size M …    ┆ Không xác định                  │
└────────────────┴─────────────────────────────────┴─────────────────────────────────┘


In [35]:
print(df_item['gender_target'].value_counts())

shape: (5, 2)
┌────────────────┬───────┐
│ gender_target  ┆ count │
│ ---            ┆ ---   │
│ str            ┆ u32   │
╞════════════════╪═══════╡
│ Sơ sinh        ┆ 3063  │
│ Không xác định ┆ 14880 │
│ Unisex         ┆ 6     │
│ Bé Gái         ┆ 5064  │
│ Bé Trai        ┆ 4319  │
└────────────────┴───────┘


In [83]:
import polars as pl

# Giả sử df_item chứa các cột 'description' và 'description_new'

# Định nghĩa các từ khóa tìm kiếm cho gender_target
gender_keywords = {
    "Bé Gái": ['bé gái', 'gái', 'nữ', 'women', 'girl'],
    "Bé Trai": ['bé trai', 'trai', 'nam', 'men', 'boy'],
    "Unisex": ['unisex','thời trang' ],
    "Sơ sinh": ['sơ sinh', 'newborn', 'tã', 'babycare', 'hóa mỹ phẩm cho bé']
}

# Hàm để xác định giới tính (gender) trong text (description hoặc description_new)
def get_gender_from_keywords(text):
    # Kiểm tra các từ khóa trong text và trả về giới tính
    found_genders = set()
    for gender, keywords in gender_keywords.items():
        if any(keyword in text for keyword in keywords):
            found_genders.add(gender)
    if "Bé Gái" in found_genders and "Bé Trai" in found_genders:
        return "Unisex"  # Nếu tìm thấy cả "Bé Gái" và "Bé Trai", gán "Unisex"
    return found_genders.pop() if found_genders else "Không xác định"  # Nếu chỉ tìm thấy một, trả về giới tính, nếu không gán "Không xác định"


# Tạo cột mới gender_target từ description và description_new mà không dùng apply
def fill_gender_target(df):
    # Kiểm tra nếu gender_target là "Không xác định"
    condition = pl.col('gender_target') == 'Không xác định'

    # Kiểm tra nếu description chứa từ khóa giới tính và thay thế
    for gender, keywords in gender_keywords.items():
        condition_description = pl.col('category').str.contains('|'.join(keywords))
        condition_description_new = pl.col('category_l1').str.contains('|'.join(keywords))
        
        # Cập nhật gender_target nếu có từ khóa trong description hoặc description_new
        df = df.with_columns(
            pl.when(condition)
            .then(
                pl.when(condition_description)
                .then(pl.lit(gender))  # Nếu tìm thấy từ khóa trong description
                .otherwise(
                    pl.when(condition_description_new)
                    .then(pl.lit(gender))  # Nếu tìm thấy từ khóa trong description_new
                    .otherwise(pl.lit("Không xác định"))
                )
            )
            .otherwise(pl.col('gender_target'))  # Giữ nguyên nếu không phải "Không xác định"
            .alias('gender_target')
        )
    
    return df

# Áp dụng hàm vào DataFrame
df_item = fill_gender_target(df_item)

# Kiểm tra kết quả
print(df_item[['gender_target', 'category', 'category_l1']].head())


shape: (5, 3)
┌────────────────┬───────────────────┬────────────────┐
│ gender_target  ┆ category          ┆ category_l1    │
│ ---            ┆ ---               ┆ ---            │
│ str            ┆ str               ┆ str            │
╞════════════════╪═══════════════════╪════════════════╡
│ Không xác định ┆ Núm ty Dr Brown   ┆ Babycare       │
│ Bé Gái         ┆ Bộ quần áo bé gái ┆ Thời trang     │
│ Không xác định ┆ Gặm nướu khác     ┆ Đồ chơi & Sách │
│ Không xác định ┆ Merries_Sơ Sinh   ┆ Tã             │
│ Không xác định ┆ Merries_Tã Quần   ┆ Tã             │
└────────────────┴───────────────────┴────────────────┘


In [84]:
print(df_item['gender_target'].value_counts())

shape: (5, 2)
┌────────────────┬───────┐
│ gender_target  ┆ count │
│ ---            ┆ ---   │
│ str            ┆ u32   │
╞════════════════╪═══════╡
│ Bé Trai        ┆ 5446  │
│ Unisex         ┆ 6     │
│ Sơ sinh        ┆ 5257  │
│ Bé Gái         ┆ 7075  │
│ Không xác định ┆ 9548  │
└────────────────┴───────┘


In [38]:
import polars as pl

# Giả sử df_item chứa các cột 'description' và 'description_new'

# Định nghĩa các từ khóa tìm kiếm cho gender_target
gender_keywords = {
    "Bé Gái": ['bé gái', 'gái', 'nữ', 'women', 'girl'],
    "Bé Trai": ['bé trai', 'trai', 'nam', 'men', 'boy'],
    "Unisex": ['unisex'],
    "Sơ sinh": ['sơ sinh', 'newborn']
}

# Hàm để xác định giới tính (gender) trong text (description hoặc description_new)
def get_gender_from_keywords(text):
    # Kiểm tra các từ khóa trong text và trả về giới tính
    found_genders = set()
    for gender, keywords in gender_keywords.items():
        if any(keyword in text for keyword in keywords):
            found_genders.add(gender)
    if "Bé Gái" in found_genders and "Bé Trai" in found_genders:
        return "Unisex"  # Nếu tìm thấy cả "Bé Gái" và "Bé Trai", gán "Unisex"
    return found_genders.pop() if found_genders else "Không xác định"  # Nếu chỉ tìm thấy một, trả về giới tính, nếu không gán "Không xác định"


# Tạo cột mới gender_target từ description và description_new mà không dùng apply
def fill_gender_target(df):
    # Kiểm tra nếu gender_target là "Không xác định"
    condition = pl.col('gender_target') == 'Không xác định'

    # Kiểm tra nếu description chứa từ khóa giới tính và thay thế
    for gender, keywords in gender_keywords.items():
        condition_description = pl.col('category_l2').str.contains('|'.join(keywords))
        condition_description_new = pl.col('category_l3').str.contains('|'.join(keywords))
        
        # Cập nhật gender_target nếu có từ khóa trong description hoặc description_new
        df = df.with_columns(
            pl.when(condition)
            .then(
                pl.when(condition_description)
                .then(pl.lit(gender))  # Nếu tìm thấy từ khóa trong description
                .otherwise(
                    pl.when(condition_description_new)
                    .then(pl.lit(gender))  # Nếu tìm thấy từ khóa trong description_new
                    .otherwise(pl.lit("Không xác định"))
                )
            )
            .otherwise(pl.col('gender_target'))  # Giữ nguyên nếu không phải "Không xác định"
            .alias('gender_target')
        )
    
    return df

# Áp dụng hàm vào DataFrame
df_item = fill_gender_target(df_item)

# Kiểm tra kết quả
print(df_item[['gender_target', 'category_l2', 'category_l3']].head())


shape: (5, 3)
┌────────────────┬────────────────────┬───────────────────────────────┐
│ gender_target  ┆ category_l2        ┆ category_l3                   │
│ ---            ┆ ---                ┆ ---                           │
│ str            ┆ str                ┆ str                           │
╞════════════════╪════════════════════╪═══════════════════════════════╡
│ Không xác định ┆ Bình sữa, phụ kiện ┆ Núm ty                        │
│ Bé Gái         ┆ Cơ cấu hàng cũ     ┆ Thời trang bé trai, bé gái cũ │
│ Không xác định ┆ 0-1Y               ┆ Gặm nướu                      │
│ Không xác định ┆ Merries            ┆ Merries                       │
│ Không xác định ┆ Merries            ┆ Merries                       │
└────────────────┴────────────────────┴───────────────────────────────┘


In [ ]:
print(df_item['gender_target'].value_counts())

shape: (5, 2)
┌────────────────┬───────┐
│ gender_target  ┆ count │
│ ---            ┆ ---   │
│ str            ┆ u32   │
╞════════════════╪═══════╡
│ Không xác định ┆ 9548  │
│ Bé Gái         ┆ 7075  │
│ Bé Trai        ┆ 5446  │
│ Unisex         ┆ 6     │
│ Sơ sinh        ┆ 5257  │
└────────────────┴───────┘


In [86]:
print(df_item['category'].unique().to_list())

['Giày chút chít 179k S15', 'Trị muỗi, côn trùng Vương Tràm Hương', 'Nón sơ sinh nhập khẩu', 'Friso Prestige Step 3', 'Giấy vệ sinh Cellox - Lốc 10', 'Aptamil Hà Lan Step 1', 'Dầu gội Enchanteur', 'Abbott Grow Step 3', 'Giày chút chít 229k S16', 'Ba lô', 'Dầu Bơ', 'President', 'Nước rửa bình sữa Arau', 'Dầu massage Nautral Botanical', 'Địu vải Kinderkraft', 'Royal Family', "Banquet D'or", 'Balo, túi cho mẹ', 'Huggies Platinum_Tã Dán M-XXL', 'Sữa tắm Lifebuoy', 'Dầu xả Dove', 'Dụng cụ bảo vệ an toàn', 'Oldenburger', 'Kem chống hăm Sudocream', 'Optimum Step 1', 'Sữa tắm Lux', 'Similac Protection Step 2', 'Tắm gội toàn thân Cetaphil', 'Quần bé trai', 'Grow Plus Xanh Step 3 Specialty', 'Xe tập đi Akigo', 'Pejoy', 'Ghế ngồi ô tô Animo', 'Hyperion', '0-12M Áo sơ sinh cài chéo tay ngắn', 'Khử mùi Avander', 'Dầu gội Cỏ Cây Hoa Lá', 'Heinz', '0-24M Áo thun bé trai tay dài đông', 'Bộ bé gái Animo Easy', 'TH True Water', 'Xe đẩy Tobby', 'Thú bông cầm tay', 'Siêu nhân, robot', 'Bubs Goat Step 3', 